## Instalar las librerías necesarias

In [0]:
%pip install requests

## Reiniciar el kernel

In [0]:
dbutils.library.restartPython()

## Importar las librerias necesarias

In [0]:
# Python
import requests
import json
import pandas as pd
import time

# PySpark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType

## 1.Configurar CoinGecko API

La API de CoinGecko no requiere autenticación, lo que la hace fácil de usar.

Usaremos el siguiente endpoint para obtener los precios actuales de criptomonedas. Endpoint:
https://api.coingecko.com/api/v3/simple/price

## 2. Consumir los datos desde la API

Creamos una función para obtener los precios actuales(USD) de las criptomonedas como Bitcoin, ethereum, etc

In [0]:
def fetch_crypto_prices(crypto_ids, vs_currency="usd"):
  url = "https://api.coingecko.com/api/v3/simple/price"
  params = {
      "ids": ",".join(crypto_ids),
      "vs_currencies": vs_currency
  }
  response = requests.get(url, params=params)

  # Manejo de errores
  if response.status_code != 200:
      raise Exception(f"Error en la API: {response.status_code}, {response.text}")
  return response.json()

In [0]:
crypto_ids = ["bitcoin", "ethereum", "dogecoin", "cardano", "solana"]
prices = fetch_crypto_prices(crypto_ids)
print(prices)

## 3. Transformar los datos en un DataFrame

Convertimos los datsos JSON en un PySpark DataFrame para procesamiento adicional

In [0]:
# Función para crear un DatFrame con los datos de los precios
def create_dataframe(prices):
    # Transformar datos JSON en una lista de filas
    rows = [
        {"crypto": key, "price_usd": value["usd"]}
        for key, value in prices.items()
    ]

    # Crear un DataFrame de pandas
    pandas_df = pd.DataFrame(rows)

    # Convertir a PySpark DataFrame
    return spark.createDataFrame(pandas_df) 

In [0]:
# Crear el DataFrame en Spark
spark_df = create_dataframe(prices)
spark_df.show()

In [0]:
display(spark_df)

## 4. Silver Layer (Guardar datos en Databricks)

Los datos pueden ser guardados como Delta Lake o CSV para su análisis posterior

In [0]:
# Guardar en formato Delta
output_path = "/tmp/crypto_prices_delta"
spark_df.write.format("delta").mode("overwrite").save(output_path)

# Confirmar guardado
print(f"Datos guardados en: {output_path}")

## 5. Gold Layer

Usaremos Databricks SQL y gráficos para analizar los datos de manera visual

In [0]:
%sql
CREATE TABLE crypto_prices
USING DELTA
LOCATION '/tmp/crypto_prices_delta'
;

In [0]:
%sql
SELECT * FROM crypto_prices
WHERE crypto = "bitcoin"
;

## 6. Agregar datos en tiempo real

Simular un flujo de data en tiempo real

In [0]:
for _ in range(10):
    prices = fetch_crypto_prices(crypto_ids)
    spark_df = create_dataframe(prices)
    spark_df_trans = spark_df
    spark_df_trans.show()

    # Guardar en formato Delta
    spark_df_trans.write.format("delta").mode("append").save(output_path)

    time.sleep(60) #C

In [0]:
display(spark_df)